In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataseet/merged_cut.csv
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/config.json
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/spiece.model
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/training_args.bin
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/tokenizer_config.json
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/model.safetensors
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/special_tokens_map.json
/kaggle/input/evmt-t5-finetune/transformers/default/1/t5_evb_translation_model/generation_config.json
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/config.json
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/spiece.model
/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model/training_ar

In [2]:
!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-1

In [3]:
from datasets import load_dataset, DatasetDict

# Load your dataset from CSV with utf-8 encoding
dataset = load_dataset(
    'csv',
    data_files='/kaggle/input/dataseet/merged_cut.csv',
    split='train',
    encoding='utf-8'
)

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
dataset[:5]

{'source': ['I have to go to sleep.',
  "Today is June 18th and it is Muiriel's birthday!",
  'Muiriel is 20 now.',
  'The password is "Muiriel".',
  'I will be back soon.'],
 'target': ['Tôi phải đi ngủ.',
  'Hôm nay là ngày 18 tháng sáu, và cũng là ngày sinh nhật của Muiriel!',
  'Bây giờ Muiriel được 20 tuổi.',
  'Mật mã là "Muiriel".',
  'Tôi trở lại mau.']}

In [5]:
dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 150000
})

In [6]:
# First, split off 10% as test set
train_val, test = dataset.train_test_split(test_size=0.1, seed=42).values()

# Then split remaining 90% into train (81%) and validation (9%)
train, validation = train_val.train_test_split(test_size=0.1, seed=42).values()

# Bundle everything into a DatasetDict
final_dataset = DatasetDict({
    'train': train,
    'validation': validation,
    'test': test
})

print(final_dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 121500
    })
    validation: Dataset({
        features: ['source', 'target'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['source', 'target'],
        num_rows: 15000
    })
})


In [7]:
# def remove_header(examples):
#     # Remove the header "translate English to Vietnamese:" from the source text
#     examples['source'] = [text.replace("translate English to Vietnamese:", "").strip() for text in examples['source']]
    
#     # If the header is also present in the target (you can modify if needed)
#     examples['target'] = [text.replace("translate English to Vietnamese:", "").strip() for text in examples['target']]
    
#     return examples

# # Apply this function to your dataset
# final_dataset = final_dataset.map(remove_header, batched=True)

# # Check the first few rows to ensure the header is removed
# print(final_dataset['train'][:5])


# Test T5 small pretrained model

In [8]:
import torch
torch.cuda.empty_cache()

In [9]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import  MT5Tokenizer, MT5ForConditionalGeneration

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "/kaggle/input/evmt-t5-finetune/transformers/epoch10_2/1/t5_evb_translation_model"
 
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)

# Move model to CUDA
model.to(device)

print(f"Model loaded on {device}")

2025-04-14 18:29:17.668373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744655357.884686      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744655357.941535      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Model loaded on cuda


In [10]:
# text = final_dataset['train'][0]['target']

# # Tokenize
# tokenized = tokenizer(text)

# # Print tokenized output
# print("Input Text:", text)
# print("Token IDs:", tokenized["input_ids"])
# print("Tokens:", [tokenizer.convert_ids_to_tokens(id) for id in tokenized["input_ids"]])

# # Decode back to text
# decoded_text = tokenizer.decode(tokenized["input_ids"], skip_special_tokens=True)
# print("Decoded Text:", decoded_text)

In [11]:
max_source_length = 256
max_target_length = 128

def preprocess_function(examples):
    # Add prefix to source text
    prefixed_source = ["" + text for text in examples['source']]
    
    # Tokenize source
    model_inputs = tokenizer(
        prefixed_source,
        max_length=max_source_length,
        padding='max_length',
        truncation=True
    )

    # Tokenize target
    labels = tokenizer(
        examples['target'],
        max_length=max_target_length,
        padding='max_length',
        truncation=True
    )['input_ids']

    # Replace padding tokens with -100 (so they are ignored in loss computation)
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in seq] for seq in labels]
    model_inputs['labels'] = labels
    
    return model_inputs

# Apply preprocessing
# tokenized_dataset = final_dataset.map(preprocess_function, batched=True)
tokenized_dataset = final_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=final_dataset["train"].column_names  # removes 'source' and 'target'
)

print(tokenized_dataset)


Map:   0%|          | 0/121500 [00:00<?, ? examples/s]

Map:   0%|          | 0/13500 [00:00<?, ? examples/s]

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 121500
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13500
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15000
    })
})


In [12]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./mt5_vietnamese_translation",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3, # change epochs
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=1000,
    report_to=["wandb"],  
    run_name="t5-small-finetune-run",
    eval_accumulation_steps=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

In [13]:
from transformers import TrainerCallback
import torch
import gc

class ClearCacheCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        gc.collect()
        torch.cuda.empty_cache()
        print("🧹 Cleared CUDA cache after epoch.")

In [14]:
from transformers import TrainerCallback

class PrintLossCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n✅ Epoch {state.epoch:.0f} Finished — Training Loss: {state.log_history[-1]['loss']}")

In [15]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,  # should be `tokenizer`, not `processing_class`
    callbacks=[PrintLossCallback(), ClearCacheCallback()]
)

/tmp/ipykernel_19/1454173417.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [16]:
!pip install wandb -q

In [17]:
import wandb

wandb.login(key="cd44b597730dc4fa6dfd5876b8f54d0ea847a004")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: phamlonghai060504 (phamlonghai060504-hanoi-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [18]:
wandb.init(project="t5_evbc_translation", name="t5-small-finetune-run")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Tracking run with wandb version 0.19.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250414_183159-ymnkogcw
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run t5-small-finetune-run
wandb: ⭐️ View project at https://wandb.ai/phamlonghai060504-hanoi-university-of-science-and-technology/t5_evbc_translation
wandb: 🚀 View run at https://wandb.ai/phamlonghai060504-hanoi-university-of-science-and-technology/t5_evbc_translation/runs/ymnkogcw


In [19]:
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.933700,0.995167
2,0.945100,0.980734
3,0.992900,0.970563



✅ Epoch 1 Finished — Training Loss: 0.9337
🧹 Cleared CUDA cache after epoch.

✅ Epoch 2 Finished — Training Loss: 0.9451
🧹 Cleared CUDA cache after epoch.

✅ Epoch 3 Finished — Training Loss: 0.9929
🧹 Cleared CUDA cache after epoch.


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=45564, training_loss=0.9416690759773513, metrics={'train_runtime': 13658.8223, 'train_samples_per_second': 26.686, 'train_steps_per_second': 3.336, 'total_flos': 9.636463116288e+16, 'train_loss': 0.9416690759773513, 'epoch': 3.0})

In [20]:
trainer.save_model('./t5_evb_translation_model')
tokenizer.save_pretrained('./t5_evb_translation_model')

('./t5_evb_translation_model/tokenizer_config.json',
 './t5_evb_translation_model/special_tokens_map.json',
 './t5_evb_translation_model/spiece.model',
 './t5_evb_translation_model/added_tokens.json')

In [21]:
# test_sentences = [
#     "Translate English to Vietnamese: gay",
#     "Translate English to Vietnamese: Hi, my name is Hai ",
#     "Translate English to Vietnamese: Hưng is a fat guy ."
# ]

# for sentence in test_sentences:
#     inputs = tokenizer(sentence, return_tensors="pt").to(model.device)
#     output = model.generate(**inputs, max_length=50)
#     print(f"Input: {sentence}")
#     output_text = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(output_text.encode('utf-8').decode())

In [22]:
# import torch
# from transformers import T5ForConditionalGeneration, T5Tokenizer
# import nltk
# from nltk.translate.bleu_score import corpus_bleu

# # Load model and tokenizer
# model_path = "./t5_evb_translation_model"
# tokenizer = T5Tokenizer.from_pretrained(model_path)
# model = T5ForConditionalGeneration.from_pretrained(model_path)
# model.eval()  # Set to evaluation mode
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [23]:
!zip -r t5_evb_translation_model.zip ./t5_evb_translation_model

  adding: t5_evb_translation_model/ (stored 0%)
  adding: t5_evb_translation_model/model.safetensors (deflated 22%)
  adding: t5_evb_translation_model/config.json (deflated 48%)
  adding: t5_evb_translation_model/special_tokens_map.json (deflated 73%)
  adding: t5_evb_translation_model/generation_config.json (deflated 29%)
  adding: t5_evb_translation_model/training_args.bin (deflated 51%)
  adding: t5_evb_translation_model/tokenizer_config.json (deflated 68%)
  adding: t5_evb_translation_model/spiece.model (deflated 46%)
